# Tutorial 2.1: One-Body Potential Energy Surface

In this section, we construct, from scratch, a 1-body MB-nrg-style PEF for CH3-NH2.

First, lets import everything we will use in this notebook.

In [8]:
import os

We will need a place to store files as we construct our PES, including input files, temporary files, and output files. Feel free to set `monomer_CH3NH2_workdir` to whichever directory you would like to use:

In [12]:
monomer_CH3NH2_workdir = "HCN_1B_PEF"
if not os.path.exists(HCN_workdir):
    os.makedirs(HCN_workdir)

Inside this directory, we will need places to put lots of files, lets define these paths here:

In [15]:
monomer_CH3NH2_initial_structure = os.path.join(monomer_CH3NH2_workdir, "initial_structure.xyz")
monomer_CH3NH2_settings = os.path.join(monomer_CH3NH2_workdir, "settings.xyz")

More information on what each of these files contains is provided as they are used in the following sections.

## 2.1.1 Input Setup

We begin by creating the two input files we will need: the initial molecular structure, and the settings file.

The initial structure file contains a "reasonable" guess for the structure of the monomer. In only needs to be good enough that when we perform a geometry optimization using DFT later we converge to the minimum structure.
MB-Fit provides the class `VESPRInitializer`, which uses basic VESPR to make guesses about the bond lengths, angles, dihedrals, etc.
This usually provides a pretty good initial guess:

In [ ]:
definition = definition_reader(monomer_CH3NH2_definition)
system_initializer = VESPRInitializer()
initial_structure = system_initializer(definition)
xyz_writer(monomer_CH3NH2_initial_structure, initial_structure)

The settings file contains some basic information about the system we are building a PEF for.

## II.1.2 Geometry Optimization

That guess is looking a little rough, lets optimize it using electronic structure theory. While the PEF will be developed at the CCSD(T) level of theory, its not important that the geometry optimization be at the same level of theory as the reference energies will be calculated at, so lets use something cheaper like a high-quality DFT functional.

In [ ]:
calculator = QChemCalculator("wb97m-v", "aug-cc-pvTZ")
slurm_job_manager = ...
slurm_job_manager.queue_job(slurm_job_template,
"""
initial_structure = xyz_reader(monomer_CH3NH2_initial_structure)
optimized_structure, optimized_energy, optimization_log_path = calculator.energy(initial_structure)
xyz_writer(monomer_CH3NH2_optimized_structure, optimized_structure)
"""
)

## II.1.? $V_{phys}$ Parametrization

Next, we will parameterize the "physical baseline" ($V_{phys}$) of the PEF.

First, we will calculate the dispersion coefficients ($C_6$) and damping parameters ($d_6$)

In [25]:
# TODO: fill this in

Next, we will calculate the atomic partial charges ($q_i$).

In [26]:
# TODO: fill this in

...

## II.1.c Normal Modes Configuration Generation

Now, we will generate the first batch of configurations for our training set. This batch will come from randomly sampling the vibrational normal modes of methylamine.

First, we need to calculate the vibrational modes of the optimized geometry:

In [ ]:
optimized_structure = xyz_reader(definition, monomer_CH3NH2_optimized_structure)
vibrational_modes, vibrational_modes_log_path = calculator.normal_modes(optimized_structure)
vibrational_modes_writer(monomer_CH3NH2_vibrational_modes, vibrational_modes)

In [ ]:
vibrational_modes_configuration_generator = NormalModesConfigurationGenerator(optimized_structure, vibrational_modes)
vibrational_modes_configs = vibrational_modes_configuration_generator.generate_configs(1000)

## II.1.? Parallel-Bias Metadynamics Configuration Generation

Let's generate the second set of configurations for our training set, those from metadynamics configurations.

## II.1.? Energy Calculation

## II.1.? Polynomial Generation

## II.1.? Polynomial Parametrization

## II.1.? Implementation in MBX

## II.1.? Test MBX Implementation

## II.1.? Run a Gas-Phase Monomer Simulation